In [1]:
import investpy
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as scs
import statsmodels.api as sm
import scipy.optimize as sco
import scipy.interpolate as sci
from pandas_datareader import data as web
import warnings
import time
import seaborn as sn

warnings.filterwarnings('ignore')
# plt.style.use('fivethirtyeight')
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'sans-serif'

In [40]:
# symbols = investpy.stocks.get_stocks_list(country='brazil')

symbols = ['ABCB4',
 'AGRO3',
 'RAIL3',
 'ALPA3',
 'ALPA4',
 'ALSO3',
 'AMAR3',
 'ABEV3',
 'ADHM3',
 'ARZZ3',
 'BBAS3',
 'BBDC3',
 'BBDC4',
 'BBRK3',
 'BEEF3',
 'BPHA3',
 'BPAN4',
 'BRAP3',
 'BRAP4',
 'BRFS3',
 'APER3',
 'BRKM3',
 'BRKM5',
 'BRML3',
 'BRPR3',
 'BRSR6',
 'OIBR3',
 'OIBR4',
 'BTOW3',
 'B3SA3',
 'CAMB4',
 'CARD3',
 'CCPR3',
 'CCRO3',
 'CEDO4',
 'CEED3',
 'CESP6',
 'CGAS5',
 'CGRA4',
 'CIEL3',
 'CMIG3',
 'CMIG4',
 'COCE5',
 'CPFE3',
 'CPLE3',
 'CPLE6',
 'CRDE3',
 'CSAN3',
 'CSMG3',
 'CSNA3',
 'LIQO3',
 'CTNM4',
 'CYRE3',
 'DASA3',
 'DIRR3',
 'DTEX3',
 'ECOR3',
 'EEEL3',
 'ELEK4',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENBR3',
 'ENGI4',
 'EQTL3',
 'YDUQ3',
 'ETER3',
 'EUCA4',
 'EVEN3',
 'EZTC3',
 'FESA4',
 'FHER3',
 'TASA4',
 'FLRY3',
 'FRIO3',
 'TIET3',
 'TIET4',
 'GFSA3',
 'GGBR3',
 'GGBR4',
 'GOAU3',
 'GOAU4',
 'GOLL4',
 'GRND3',
 'GSHP3',
 'HBOR3',
 'HGTX3',
 'PRIO3',
 'HYPE3',
 'IDNT3',
 'IDVL4',
 'IGTA3',
 'MEAL3',
 'INEP3',
 'INEP4',
 'ITSA3',
 'ITSA4',
 'ITUB3',
 'ITUB4',
 'JBSS3',
 'JFEN3',
 'JHSF3',
 'JSLG3',
 'KEPL3',
 'KLBN4',
 'LAME3',
 'LAME4',
 'LEVE3',
 'LIGT3',
 'LLIS3',
 'LOGN3',
 'LPSB3',
 'LREN3',
 'LUPA3',
 'MDIA3',
 'MGEL4',
 'MGLU3',
 'MILS3',
 'MMXM3',
 'ENEV3',
 'MRFG3',
 'MRVE3',
 'MTIG4',
 'MULT3',
 'MYPK3',
 'NATU3',
 'ODPV3',
 'OSXB3',
 'PCAR4',
 'PDGR3',
 'PETR3',
 'PETR4',
 'PFRM3',
 'PINE4',
 'PLAS3',
 'PMAM3',
 'POMO3',
 'POMO4',
 'POSI3',
 'PSSA3',
 'PTBL3',
 'PTNT4',
 'ENAT3',
 'QUAL3',
 'RAPT3',
 'RAPT4',
 'RDNI3',
 'RENT3',
 'FRTA3',
 'RNEW11',
 'ROMI3',
 'RSID3',
 'SANB11',
 'SBSP3',
 'SCAR3',
 'SGPS3',
 'SHOW3',
 'SLCE3',
 'SLED4',
 'SMTO3',
 'SULA11',
 'EGIE3',
 'TCSA3',
 'TECN3',
 'TEKA4',
 'TGMA3',
 'TIMP3',
 'TOTS3',
 'TPIS3',
 'TRIS3',
 'TAEE11',
 'TRPL4',
 'TRPN3',
 'UGPA3',
 'UNIP3',
 'UNIP6',
 'USIM3',
 'USIM5',
 'TESA3',
 'VALE3',
 'VIVR3',
 'VIVT3',
 'VIVT4',
 'VLID3',
 'WEGE3',
 'BRSR5',
 'CESP5',
 'ELPL3',
 'SANB4',
 'STBP3',
 'CCXC3',
 'CESP3',
 'CGRA3',
 'CLSC4',
 'COCE3',
 'TASA3',
 'COGN3',
 'LCAM3',
 'RADL3',
 'UCAS3',
 'ALUP11',
 'ANIM3',
 'BBSE3',
 'BSEV3',
 'CPRE3',
 'CVCB3',
 'LINX3',
 'SEER3',
 'SMLS3',
 'TUPY3',
 'SQIA3',
 'BIOM3',
 'KLBN11',
 'NUTR3',
 'BAZA3',
 'HGBS11',
 'HGPO11',
 'MBRF11',
 'TCNO3',
 'ABCP11',
 'RBED11',
 'BBPO11',
 'BBRC11',
 'BBVJ11',
 'BPFF11',
 'BRCR11',
 'CTXT11',
 'CXTL11',
 'EURO11',
 'RCRB11',
 'FIGS11',
 'FIXX11',
 'FLMA11',
 'FPAB11',
 'HMOC11',
 'HGCR11',
 'HGLG11',
 'HGRE11',
 'JSRE11',
 'KNCR11',
 'KNRE11',
 'KNRI11',
 'MXRF11',
 'ONEF11',
 'PQDP11',
 'PRSV11',
 'RBBV11',
 'RBCB11',
 'RBGS11',
 'RBRD11',
 'RBVO11',
 'RDES11',
 'RNDP11',
 'RNGO11',
 'SAAG11',
 'SDIL11',
 'SHPH11',
 'SPTW11',
 'TBOF11',
 'TRXL11',
 'VLOL11',
 'VRTA11',
 'XPCM11',
 'XTED11',
 'SANB3',
 'SEDU3',
 'FRAS3',
 'GEPA4',
 'OFSA3',
 'RLOG3',
 'TOYB3',
 'AAPL34',
 'ABTT34',
 'AMGN34',
 'AMZO34',
 'ARMT34',
 'ATTB34',
 'AXPB34',
 'BERK34',
 'BOAC34',
 'CATP34',
 'CMCS34',
 'COCA34',
 'CSCO34',
 'DISB34',
 'EXXO34',
 'FDMO34',
 'GEOO34',
 'GSGI34',
 'HALI34',
 'HOME34',
 'HPQB34',
 'ITLC34',
 'JNJB34',
 'JPMC34',
 'LMTB34',
 'MMMC34',
 'MRCK34',
 'MSCD34',
 'MSFT34',
 'NIKE34',
 'ORCL34',
 'PFIZ34',
 'PGCO34',
 'SBUB34',
 'VERZ34',
 'VISA34',
 'WALM34',
 'WFCO34',
 'WIZS3',
 'BOEI34',
 'CHVX34',
 'COPH34',
 'CTGP34',
 'FBOK34',
 'IBMB34',
 'LILY34',
 'MCDC34',
 'MFII11',
 'MSBR34',
 'NFLX34',
 'PEPB34',
 'SLBG34',
 'UPSS34',
 'USBC34',
 'GUAR3',
 'ACNB34',
 'BONY34',
 'DHER34',
 'IGBR3',
 'KMBB34',
 'METB34',
 'TELB4',
 'TGTB34',
 'WHRL3',
 'AVON34',
 'BMYB34',
 'COLG34',
 'COTY34',
 'EBAY34',
 'FCXO34',
 'FDXB34',
 'GDBR34',
 'HSHY34',
 'MDLZ34',
 'MOSC34',
 'QCOM34',
 'TEXA34',
 'TIFF34',
 'UTEC34',
 'WUNI34',
 'XRXB34',
 'SHUL4',
 'ATOM3',
 'KHCB34',
 'TIET11',
 'SAPR4',
 'CRIV4',
 'RPMG3',
 'BRSR3',
 'DOMC11',
 'ENGI11',
 'SNSY5',
 'DMMO3',
 'ABCB10',
 'ANIM3T',
 'AZEV4',
 'BAHI3',
 'BALM4',
 'BAUH4',
 'BDLL4',
 'BEES3',
 'BEES4',
 'BGIP4',
 'BMEB3',
 'BMEB4',
 'BMIN3',
 'BMIN4',
 'BMKS3',
 'BNBR3',
 'BOBR4',
 'BRIV3',
 'BRIV4',
 'BSLI4',
 'BTTL3',
 'CBEE3',
 'CEEB3',
 'CEEB5',
 'CELP3',
 'CELP5',
 'CEPE5',
 'CGAS3',
 'CPRE3T',
 'CRIV3',
 'CRPG5',
 'CRPG6',
 'CSRN3',
 'CTKA4',
 'CTNM3',
 'CTSA3',
 'CTSA4',
 'CVCB3T',
 'DOHL4',
 'DTCY3',
 'EALT4',
 'EKTR4',
 'ELEK3',
 'EMAE4',
 'ENGI3',
 'ENMT3',
 'ESTR4',
 'FESA3',
 'FNAM11',
 'FSRF11',
 'GPCP3',
 'HAGA4',
 'HBTS5',
 'HETA4',
 'HOOT4',
 'IDVL3',
 'JBDU3',
 'JBDU4',
 'JOPA3',
 'JOPA4',
 'KLBN3',
 'MEND5',
 'MNDL3',
 'MNPR3',
 'MTSA4',
 'MWET4',
 'DMMO3T',
 'PEAB4',
 'PNVL3',
 'PNVL4',
 'RANI3',
 'RANI4',
 'RCSL3',
 'RCSL4',
 'REDE3',
 'RPAD5',
 'RPAD6',
 'SEER3T',
 'WLMM3',
 'WLMM4',
 'SLED3',
 'SMLS3T',
 'SOND5',
 'SOND6',
 'SPRI3',
 'TCNO4',
 'TELB3',
 'TOYB4',
 'TRPL3',
 'UNIP5',
 'USIM6',
 'VULC3',
 'VVAR3',
 'WHRL4',
 'ALMI11',
 'ANCR11B',
 'ATSA11B',
 'BBFI11B',
 'BCFF11',
 'BMLC11B',
 'CEOC11',
 'CNES11',
 'MRSA6BF',
 'CPTS11B',
 'CXCE11B',
 'DRIT11B',
 'EDFO11B',
 'EDGA11',
 'ENMA3B',
 'FAED11',
 'FAMB11B',
 'FCFL11',
 'FEXC11',
 'FIIP11B',
 'FLRP11',
 'FVBI11',
 'HCRI11',
 'HTMX11',
 'JRDM11',
 'MAXR11',
 'NSLU11',
 'TRNT11',
 'WPLZ11B',
 'ARNC34',
 'ABBV34',
 'BBYY34',
 'BCRI11',
 'BIIB34',
 'BLAK34',
 'CHKE34',
 'COWC34',
 'CTSH34',
 'CVSH34',
 'CXRI11',
 'DEAI34',
 'FSLR34',
 'GILD34',
 'GOGL34',
 'GOGL35',
 'GPSI34',
 'JCPC34',
 'MACY34',
 'MDTC34',
 'RIGG34',
 'ROST34',
 'SANC34',
 'SCHW34',
 'SPRN34',
 'SSFO34',
 'TMOS34',
 'TRVC34',
 'USSX34',
 'VLOE34',
 'WSON33',
 'AALR3',
 'AFLT3',
 'BGIP3',
 'BNFS11',
 'CBOP11',
 'CEBR3',
 'CEDO3',
 'ESUD11',
 'ESUT11',
 'FIIB11',
 'FIVN11',
 'FMOF11',
 'FNOR11',
 'FSPE11',
 'FSTU11',
 'GPAR3',
 'GRLV11',
 'MAPT3',
 'MMXM11',
 'PABY11',
 'PATI4',
 'PLRI11',
 'PORD11',
 'RBDS11',
 'RPAD3',
 'SCPF11',
 'VSPT3',
 'XPOM11',
 'POMO10',
 'AIGB34',
 'DUKB34',
 'MOVI3',
 'PARD3',
 'BPAC11',
 'PPLA11',
 'AALL34',
 'GMCO34',
 'LBRN34',
 'TWTR34',
 'UPAC34',
 'RNEW3',
 'RNEW4',
 'AZUL4',
 'KNIP11',
 'TEND3',
 'TSLA34',
 'CRFB3',
 'GBIO33',
 'OMGE3',
 'IRBR3',
 'GGRC11',
 'GEPA3',
 'SAPR3',
 'CAML3',
 'VISC11',
 'SUZB3',
 'SAPR11',
 'BRDT3',
 'NEOE3',
 'BKBR3',
 'FMXB34',
 'CLGN34',
 'HONB34',
 'ALZR11',
 'GFSA1',
 'DDNB34',
 'UBSG34',
 'TXRX4',
 'OUJP11',
 'BCIA11',
 'BOXP34',
 'GPRO34',
 'MSPA3',
 'HFOF11',
 'RBRF11',
 'HAPV3',
 'GNDI3',
 'BIDI4',
 'JPSA3',
 'GPIV33',
 'IRDM11',
 'FOFT11',
 'THRA11',
 'TAEE4',
 'BIDI4T',
 'XPLG11',
 'RBRR11',
 'WPLZ11',
 'MGFF11',
 'TGAR11',
 'HGRU11',
 'ALUP3',
 'ALUP4',
 'LOGG3',
 'KNHY11',
 'OUCY11',
 'FVPQ11',
 'TASA13',
 'TASA15',
 'TASA17',
 'CARE11',
 'TAEE3',
 'XPIN11',
 'VSHO11',
 'DMMO11',
 'AZEV3',
 'AHEB3',
 'BALM3',
 'BPAC3',
 'EALT3',
 'ECPR3',
 'HAGA3',
 'ITEC3',
 'LIPR3',
 'PATI3',
 'PEAB3',
 'SULA3',
 'BPAC5',
 'BRGE11',
 'BRGE12',
 'BRGE6',
 'CELP6',
 'CELP7',
 'CORR4',
 'EEEL4',
 'ENMT4',
 'LUXM4',
 'MEND6',
 'MERC4',
 'RSUL4',
 'SULA4',
 'BEEF11',
 'JBDU1',
 'JBDU2',
 'LOGN12',
 'MYPK12',
 'PATC11',
 'BBDC3',
 'VGIR11',
 'VILG11',
 'CNTO3',
 'BTCR11',
 'CAMB10',
 'IDVL9',
 'MELI34',
 'MOAR3',
 'RBVA11',
 'MRSA3B',
 'VTLT11',
 'GTWR11',
 'BIDI11',
 'BIDI3',
 'DMAC11',
 'TFOF11',
 'HCTR11',
 'TIET2',
 'HSML11',
 'XPHT11',
 'XPHT12',
 'HABT11',
 'RECT11',
 'AQLL11',
 'ARFI11B',
 'BARI11',
 'BPRP11',
 'DOVL11B',
 'FISC11',
 'FTCE11B',
 'HBTT11',
 'HGFF11',
 'HUSC11',
 'JPPC11',
 'KINP11',
 'LATR11B',
 'NPAR11',
 'NVHO11',
 'ORPD11',
 'OULG11B',
 'PRSN11B',
 'RBFF11',
 'RBRP11',
 'RBRY11',
 'RCFA11',
 'RDPD11',
 'REIT11',
 'SADI11',
 'SAIC11B',
 'TOUR11',
 'VERE11',
 'WTSP11B',
 'YCHY11',
 'VIVA3',
 'CEAB3',
 'BMGB11',
 'BMGB4',
 'SDIP11',
 'IBFF11',
 'CVBI11',
 'HRDF11']


print(len(symbols))

symbols = ['ABCB4',
 'AGRO3',
 'RAIL3',
 'ALPA3',
 'ALPA4',
 'ALSO3',
 'AMAR3',
 'ABEV3',
 'ADHM3']

717


In [41]:
initial = time.time()

# Select Dates
from_date = '01/01/2014'
to_date = datetime.today().strftime('%d/%m/%Y') # Today
# to_date = '06/09/2020'

# Get all stocks in one DF
data = []
new_symbols = [] 

for i in symbols:
    try:
        data.append(investpy.get_stock_historical_data(
                                             stock=i,
                                             country='brazil',
                                             from_date=from_date, 
                                             to_date=to_date, 
                                             interval='Daily'))
        new_symbols.append(i)
        
    except:
        pass

print("It took {:.2f} seconds".format(time.time() - initial))

It took 26.09 seconds


In [42]:
# Number of stocks = Number of Dataframes !!
dataf = [data[i] for i in range(len(new_symbols))] 

# Concatenate all stocks data
df = pd.concat(objs = dataf,axis = 1)
columns = [['Open_' + str(i) ,'High_' + str(i), 'Low_' + str(i), str(i), 'Volume_' + str(i) ,'Curreny_' + str(i)] for i in symbols]

# Convert columns list into a flat list 
col = [ii for i in columns for ii in i]

print(df.columns)

# # Rename columns in DF
df.columns = col

# # Select Price Column for each stock 
close_price = [i for i in symbols]
df = df[close_price]
df.to_csv('brazilian_stocks.csv') 
# dg = df

df.dtypes

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Currency', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Currency', 'Open', 'High', 'Low', 'Close',
       'Volume', 'Currency', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Currency', 'Open', 'High', 'Low', 'Close', 'Volume', 'Currency',
       'Open', 'High', 'Low', 'Close', 'Volume', 'Currency', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Currency', 'Open', 'High', 'Low', 'Close',
       'Volume', 'Currency', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Currency'],
      dtype='object')


ABCB4    float64
AGRO3    float64
RAIL3    float64
ALPA3    float64
ALPA4    float64
ALSO3    float64
AMAR3    float64
ABEV3    float64
ADHM3    float64
dtype: object

In [43]:
# df = pd.read_csv("brazilian_stocks.csv", parse_dates=True,
#                  verbose=True)
 
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)


df.head
# for code in symbols:
#       df[code] = pd.to_numeric(df[code], downcast='float')
# df.head()

<bound method NDFrame.head of             ABCB4  AGRO3  RAIL3  ALPA3  ALPA4  ALSO3  AMAR3
Date                                                       
2014-01-02   7.44   9.50  21.79    NaN   8.09  17.18  16.53
2014-01-03   7.58   9.21  21.97   9.63   8.01  17.24  16.89
2014-01-06   7.66   9.27  21.93   9.66   7.89  17.13  16.89
2014-01-07   7.66   9.39  21.00   9.84   7.65  17.32  16.62
2014-01-08   7.84   9.49  21.62   9.66   7.66  17.41  16.66
...           ...    ...    ...    ...    ...    ...    ...
2020-09-08  13.75  21.02  22.06  33.51  37.19  28.42   8.21
2020-09-09  13.53  21.07  22.08  33.41  37.15  27.78   8.16
2020-09-10  13.22  21.26  21.45  33.00  36.89  27.17   8.05
2020-09-11  12.94  21.37  21.02  34.49  37.04  26.81   7.75
2020-09-14  13.28  22.06  21.30  33.58  37.42  27.83   8.18

[1658 rows x 7 columns]>

In [44]:
# Log Return
log_ret = np.log(df/df.shift(1))
log_ret.dropna(inplace=True)

# Simple Return
simple_ret = df.pct_change()
simple_ret.dropna(inplace=True)

In [ ]:
# Volatility Function
def realized_volatility(x):
    return np.sqrt(np.sum(x**2))

# Resample Data and Apply Function
freq= 'D'
n_days = 252
r_vol = log_ret.groupby(pd.Grouper(freq=freq)).apply(realized_volatility) # set frequency
r_vol = r_vol*np.sqrt(n_days) # change window


In [ ]:
# Statistics
def print_statistics(a1):
  
    sta1 = scs.describe(a1)

    print('%14s %14s' % ('statistic','value')) 
    print(45*"-") 
    print('%14s %14.3f' % ('size',sta1[0])) 
    print('%14s %14.3f' % ('min',sta1[1][0])) 
    print('%14s %14.3f' % ('max',sta1[1][1]))
    print('%14s %14.3f' % ('mean',sta1[2]))  
    print('%14s %14.3f' % ('std',np.sqrt(sta1[3])))
    print('%14s %14.3f' % ('skew',sta1[4])) 
    print('%14s %14.3f' % ('kurtosis',sta1[5])) 

# Normatity Test
def normality_tests(arr):

    print("Skew of data set  %14.3f" % scs.skew(arr))
    print("Skew test p-value %14.3f" % scs.skewtest(arr)[1])
    print("Kurt of data set  %14.3f" % scs.kurtosis(arr))
    print("Kurt test p-value %14.3f" % scs.kurtosistest(arr)[1])
    print("Norm test p-value %14.3f" % scs.normaltest(arr)[1])

# Monte Carlo Simuation

In [ ]:
start = time.time()

# Simple Return
returns_annual = simple_ret.mean() * n_days

# covariance
cov_daily = simple_ret.cov()
cov_annual = cov_daily * n_days

# empty lists to store returns, volatility and weights of imiginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# number of combinations for imaginary portfolios
num_stocks = len(symbols)
num_portfolios = 10**6

# set random seed for reproduction
np.random.seed(42)

# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):

    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    sharpe = returns / volatility # riskfree = 0
    sharpe_ratio.append(sharpe) 
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

# extend original dictionary to accomodate each stock and weight in the portfolio
for counter,symbol in enumerate(symbols):
    portfolio[symbol+' Weight'] = [Weight[counter] for Weight in stock_weights]


print(f"It took {time.time() - start} seconds")
    
portfolio.keys()

In [ ]:
# make a nice dataframe of the extended dictionary
dfm = pd.DataFrame(portfolio)

# find min Volatility & max sharpe values in the dataframe (df)
min_volatility = dfm['Volatility'].min()
max_sharpe = dfm['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
sharpe_portfolio = dfm.loc[dfm['Sharpe Ratio'] == max_sharpe]
min_variance_port = dfm.loc[dfm['Volatility'] == min_volatility]

# plotting frontier
fig,ax1=plt.subplots()
dfm.plot.scatter('Volatility', 'Returns', c='Sharpe Ratio', 
                cmap='RdYlBu', figsize=(10, 8), grid=True,ax=ax1)
plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], marker=(5,1,0), c='y',s=500)
plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], marker=(5,1,0), c='r', s=500 )
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier',fontsize=20)
plt.show()
plt.savefig("Efficient_Frontier.png")

In [ ]:

print('Minimun Variance Portfolio:\n')
min_volatility = dfm['Volatility'].min()
min_variance_port = dfm.loc[dfm['Volatility'] == min_volatility]

# print(type(min_variance_port))
 
# min_variance_port.sort_values(by=['Brand'], inplace=True)

# variance_result.write('Minimun Variance Portfolio:\n')

# min_variance_port1 = min_variance_port1.to_numpy()

print(min_variance_port.T)
# min_variance_port1 = min_variance_port1.sort()
# min_variance_port1.savetxt('minimum_variance_portifolio.txt') 

#
array = min_variance_port.to_numpy()

file = open("minimum_variance_portifolio.txt", "w")
buffer = "Returns: "  +     str(array[0][0])   + "\nVolatility: "  +    str(array[0][1]) + "\nSharpe Ratio: "  +  str(array[0][2]) + "\n"

file.write(buffer)
for i in range(len(symbols) - 1):
    buffer =  str(symbols[i]) + "," + str(array[0][i + 3]) + "\n"
    file.write(buffer)
file.close()     
    
# np.savetxt('minimum_variance_portifolio.txt', min_variance_port1.T)
# for i in min_variance_port.iterrows(): 
#     variance_result.write(str(i) + "\n")


# print('\n\nMaximum sharpe Portfolio:\n')
# print(sharpe_portfolio.T)

# variance_result.write('Maximum sharpe Portfolio:\n')
# for i in range(len(min_variance_port)):
#     variance_result.write(min_variance_port[i].T + "\n")
    

# variance_result.close()

# Optimization Problem



In [ ]:
# Functions for Portfolio Calculations
def statistics(weights):
    ''' Sharpe ratio for riskfree = 0
    '''
    weights = np.array(weights)
    p_rets = np.sum(simple_ret.mean() * weights) * n_days
    p_volt = np.sqrt(np.dot(weights.T, np.dot(simple_ret.cov() * n_days, weights)))
    return np.array([p_rets, p_volt, p_rets / p_volt])

In [ ]:
# Functions to Minimize
def min_func_sharpe(weights):
    return -statistics(weights)[2]

def min_func_variance(weights):
    return statistics(weights)[1]

In [ ]:
# Constraints
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1}) # No Short positions

# Boundaries
bnds = tuple((0, 1) for x in range(num_stocks))

# Equal Weights
equal_weights = num_stocks*[1./num_stocks]

In [ ]:
# start = time.time()

# # Maximization of Sharpe Ratio 
# opts = sco.minimize(min_func_sharpe, equal_weights, method='SLSQP',
#                        bounds=bnds, constraints=cons)

# print("It took {:.2f} seconds".format(time.time() - start))
# print(opts)

In [ ]:
# # # Optimal weights that maximizes the sharpe ratio
# pd.DataFrame([round(x,4) for x in opts['x']],index=symbols).T

In [ ]:
# # Optimal Portfolio Metrics [Annual Return, Annual Volatility, Sharpe Ratio]
# statistics(opts['x']).round(5)

In [ ]:
# # # Minimun Variance Portfolio
# start = time.time()

# optv = sco.minimize(min_func_variance, equal_weights, method='SLSQP',
#                        bounds=bnds, constraints=cons)

# print("It took {:.2f} seconds".format(time.time() - start))
# optv

In [ ]:
# # Optimal Weights in Minumum Variance Portfolio
# pd.DataFrame([round(x,2) for x in optv['x']],index=symbols).T

In [ ]:
# # Minimum Variance Porfolio Metrics [Annual Return, Annual Volatility, Sharpe Ratio]
# statistics(optv['x']).round(5)

In [ ]:
# # Eficient Frontier Calculations

# cons = ({'type': 'eq', 'fun': lambda x:  statistics(x)[0] - i},
#         {'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})

# bnds = tuple((0, 1) for x in weights)

# target_ret = np.linspace(0.02, 0.142, 50)
# target_vol = []

# for i in target_ret:
#     res = sco.minimize(min_func_variance, equal_weights, method='SLSQP',
#                        bounds=bnds, constraints=cons)
#     target_vol.append(res['fun'])

# target_vol = np.array(target_vol)

In [ ]:
# # Plotting frontier
# fig,ax1=plt.subplots()
# # Random Portfolio Composition
# dfm.plot.scatter('Volatility', 'Returns', c='Sharpe Ratio', 
#                 cmap='RdYlBu', figsize=(10, 8), grid=True,ax=ax1)
# # Eficient Frontier
# plt.plot(target_vol, target_ret, c='black', lw=2, linestyle='--')
# # portfolio with highest Sharpe ratio
# # plt.plot(statistics(opts['x'])[1], statistics(opts['x'])[0], 'r*', markersize=28, c='y')    
# # minimum variance portfolio
# plt.plot(statistics(optv['x'])[1], statistics(optv['x'])[0], 'y*', markersize=28, c='r')

# # Plot Stocks
# MARKS = ['o', 'X', 'd', '*','x']

# for i in range(num_stocks):
#     plt.scatter(x=np.sqrt(cov_annual.iloc[i, i]),y=returns_annual[i],marker=MARKS[i],
#              s=150, color='black', label=symbols[i])
# plt.legend(loc = 'upper left')

# plt.xlabel('Volatility (Std. Deviation)')
# plt.ylabel('Expected Returns')
# plt.title('Efficient Frontier',fontsize=20)
# plt.show()

In [ ]:
# Pyportfolio 


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Portfolio Optimization

mu = expected_returns.mean_historical_return(df)
s = risk_models.sample_cov(df)

# Optimize for max Sharpe Ratio

ef = EfficientFrontier(mu,s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights,'\n')

# Riskfree = 0
ef.portfolio_performance(verbose=True, risk_free_rate=0)

In [ ]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

total_position = 15000

# Take the last price of stocks
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = total_position)

allocation, leftover = da.lp_portfolio()
print("Discrete Allocation:", allocation)
print('Funds Remaining: ${:.2f}'.format(leftover))

In [ ]:
print("It took {:.2f} seconds" .format(time.time() - initial))